# HeyAuto_Challenge

### Setup config and read data

In [63]:
# Import Libraries
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [65]:
# Setup the Configuration
spark = SparkSession.builder.appName("HeyAuto_Challenge").getOrCreate()

In [66]:
heyauto_csv = "data_heyAuto.csv"

heyauto_df = spark.read.format("csv")\
                                .option("inferSchema", "true")\
                                .option("header", "true")\
                                .load(heyauto_csv)

In [67]:
heyauto_df.show()

+-----------------+---------+--------+----+------------+---------+
|              vin|     make|   model|year|        trim|Condition|
+-----------------+---------+--------+----+------------+---------+
|2T1FF28P81C438164|    Acura|      TL|1956|       Sport|        U|
|1C4NJPFA8ED519678|    Honda|   Civic|2020|       Sport|        U|
|YV4960DZ8A2080987|   Toyota|    null|2019|        Base|        U|
|3C3CFFAR3DT571989|     Audi|      A4|2022|         300|        N|
|5FNYF4H5XFB054084|     Ford|   F-150|2025|        null|        N|
|JM1BM1U79E1108676|Chevrolet|     300|2007|        null|        U|
|5TDZA23C46S533612|      BMW|      M4|null|        null|        N|
|JTEBU5JR8A5024764|  Hyundai|Santa Fe|2011|Trim-Package|        U|
|2G4WS52J121278965|     Cart|     200|2021|       Sport|        N|
| 5TDZA3EH4BS08656|   Nissan|   Rogue|2022|        null|        N|
|2G4WS52J121278966|     null|    null|null|        null|     null|
+-----------------+---------+--------+----+------------+------

### Condition Transformation U/N -> Used/New

In [69]:
heyauto_transformed_df = heyauto_df.withColumn("Condition", F.when(heyauto_df.Condition == "U", "Used")\
                                                .when(heyauto_df.Condition == "N", "New")\
                                                .otherwise(None))

In [70]:
heyauto_transformed_df.show()

+-----------------+---------+--------+----+------------+---------+
|              vin|     make|   model|year|        trim|Condition|
+-----------------+---------+--------+----+------------+---------+
|2T1FF28P81C438164|    Acura|      TL|1956|       Sport|     Used|
|1C4NJPFA8ED519678|    Honda|   Civic|2020|       Sport|     Used|
|YV4960DZ8A2080987|   Toyota|    null|2019|        Base|     Used|
|3C3CFFAR3DT571989|     Audi|      A4|2022|         300|      New|
|5FNYF4H5XFB054084|     Ford|   F-150|2025|        null|      New|
|JM1BM1U79E1108676|Chevrolet|     300|2007|        null|     Used|
|5TDZA23C46S533612|      BMW|      M4|null|        null|      New|
|JTEBU5JR8A5024764|  Hyundai|Santa Fe|2011|Trim-Package|     Used|
|2G4WS52J121278965|     Cart|     200|2021|       Sport|      New|
| 5TDZA3EH4BS08656|   Nissan|   Rogue|2022|        null|      New|
|2G4WS52J121278966|     null|    null|null|        null|     null|
+-----------------+---------+--------+----+------------+------

### Validation Process

In [71]:
heyauto_val_df = heyauto_transformed_df.withColumn("validationTest", F.array())

In [72]:
heyauto_val_df.show()

+-----------------+---------+--------+----+------------+---------+--------------+
|              vin|     make|   model|year|        trim|Condition|validationTest|
+-----------------+---------+--------+----+------------+---------+--------------+
|2T1FF28P81C438164|    Acura|      TL|1956|       Sport|     Used|            []|
|1C4NJPFA8ED519678|    Honda|   Civic|2020|       Sport|     Used|            []|
|YV4960DZ8A2080987|   Toyota|    null|2019|        Base|     Used|            []|
|3C3CFFAR3DT571989|     Audi|      A4|2022|         300|      New|            []|
|5FNYF4H5XFB054084|     Ford|   F-150|2025|        null|      New|            []|
|JM1BM1U79E1108676|Chevrolet|     300|2007|        null|     Used|            []|
|5TDZA23C46S533612|      BMW|      M4|null|        null|      New|            []|
|JTEBU5JR8A5024764|  Hyundai|Santa Fe|2011|Trim-Package|     Used|            []|
|2G4WS52J121278965|     Cart|     200|2021|       Sport|      New|            []|
| 5TDZA3EH4BS086

##### Validation (hasMake)

In [73]:
heyauto_val1_df = heyauto_val_df.withColumn("validationTest", 
                                                F.when(F.expr("make is NULL"), F.array_union(F.col("validationTest"), F.array(F.lit("hasNotMake"))))\
                                                .otherwise(F.col("validationTest")))

In [74]:
heyauto_val1_df.show(truncate=False)

+-----------------+---------+--------+----+------------+---------+--------------+
|vin              |make     |model   |year|trim        |Condition|validationTest|
+-----------------+---------+--------+----+------------+---------+--------------+
|2T1FF28P81C438164|Acura    |TL      |1956|Sport       |Used     |[]            |
|1C4NJPFA8ED519678|Honda    |Civic   |2020|Sport       |Used     |[]            |
|YV4960DZ8A2080987|Toyota   |null    |2019|Base        |Used     |[]            |
|3C3CFFAR3DT571989|Audi     |A4      |2022|300         |New      |[]            |
|5FNYF4H5XFB054084|Ford     |F-150   |2025|null        |New      |[]            |
|JM1BM1U79E1108676|Chevrolet|300     |2007|null        |Used     |[]            |
|5TDZA23C46S533612|BMW      |M4      |null|null        |New      |[]            |
|JTEBU5JR8A5024764|Hyundai  |Santa Fe|2011|Trim-Package|Used     |[]            |
|2G4WS52J121278965|Cart     |200     |2021|Sport       |New      |[]            |
|5TDZA3EH4BS0865

##### Validation (hasModel)

In [75]:
heyauto_val2_df = heyauto_val1_df.withColumn("validationTest", 
                                                F.when(F.expr("model is NULL"), F.array_union(F.col("validationTest"), F.array(F.lit("hasNotModel"))))\
                                                .otherwise(F.col("validationTest")))

In [76]:
heyauto_val2_df.show(truncate=False)

+-----------------+---------+--------+----+------------+---------+-------------------------+
|vin              |make     |model   |year|trim        |Condition|validationTest           |
+-----------------+---------+--------+----+------------+---------+-------------------------+
|2T1FF28P81C438164|Acura    |TL      |1956|Sport       |Used     |[]                       |
|1C4NJPFA8ED519678|Honda    |Civic   |2020|Sport       |Used     |[]                       |
|YV4960DZ8A2080987|Toyota   |null    |2019|Base        |Used     |[hasNotModel]            |
|3C3CFFAR3DT571989|Audi     |A4      |2022|300         |New      |[]                       |
|5FNYF4H5XFB054084|Ford     |F-150   |2025|null        |New      |[]                       |
|JM1BM1U79E1108676|Chevrolet|300     |2007|null        |Used     |[]                       |
|5TDZA23C46S533612|BMW      |M4      |null|null        |New      |[]                       |
|JTEBU5JR8A5024764|Hyundai  |Santa Fe|2011|Trim-Package|Used     |[]  

##### Validation (hasYear)

In [77]:
heyauto_val3_df = heyauto_val2_df.withColumn("validationTest", 
                                                F.when(F.expr("year is NULL"), F.array_union(F.col("validationTest"), F.array(F.lit("hasNotYear"))))\
                                                .otherwise(F.col("validationTest")))

In [78]:
heyauto_val3_df.show(truncate=False)

+-----------------+---------+--------+----+------------+---------+-------------------------------------+
|vin              |make     |model   |year|trim        |Condition|validationTest                       |
+-----------------+---------+--------+----+------------+---------+-------------------------------------+
|2T1FF28P81C438164|Acura    |TL      |1956|Sport       |Used     |[]                                   |
|1C4NJPFA8ED519678|Honda    |Civic   |2020|Sport       |Used     |[]                                   |
|YV4960DZ8A2080987|Toyota   |null    |2019|Base        |Used     |[hasNotModel]                        |
|3C3CFFAR3DT571989|Audi     |A4      |2022|300         |New      |[]                                   |
|5FNYF4H5XFB054084|Ford     |F-150   |2025|null        |New      |[]                                   |
|JM1BM1U79E1108676|Chevrolet|300     |2007|null        |Used     |[]                                   |
|5TDZA23C46S533612|BMW      |M4      |null|null        

##### Validation (hasCondition)

In [79]:
heyauto_val4_df = heyauto_val3_df.withColumn("validationTest", 
                                                F.when(F.expr("Condition is NULL"), F.array_union(F.col("validationTest"), F.array(F.lit("hasNotCondition"))))\
                                                .otherwise(F.col("validationTest")))

In [80]:
heyauto_val4_df.show(truncate=False)

+-----------------+---------+--------+----+------------+---------+------------------------------------------------------+
|vin              |make     |model   |year|trim        |Condition|validationTest                                        |
+-----------------+---------+--------+----+------------+---------+------------------------------------------------------+
|2T1FF28P81C438164|Acura    |TL      |1956|Sport       |Used     |[]                                                    |
|1C4NJPFA8ED519678|Honda    |Civic   |2020|Sport       |Used     |[]                                                    |
|YV4960DZ8A2080987|Toyota   |null    |2019|Base        |Used     |[hasNotModel]                                         |
|3C3CFFAR3DT571989|Audi     |A4      |2022|300         |New      |[]                                                    |
|5FNYF4H5XFB054084|Ford     |F-150   |2025|null        |New      |[]                                                    |
|JM1BM1U79E1108676|Chevr

##### Validation (validVin)

In [81]:
heyauto_val5_df = heyauto_val4_df.withColumn("validationTest", 
                                                F.when(F.length("vin") != 17, F.array_union(F.col("validationTest"), F.array(F.lit("notValidVin"))))\
                                                .otherwise(F.col("validationTest")))

In [82]:
heyauto_val5_df.show(truncate=False)

+-----------------+---------+--------+----+------------+---------+------------------------------------------------------+
|vin              |make     |model   |year|trim        |Condition|validationTest                                        |
+-----------------+---------+--------+----+------------+---------+------------------------------------------------------+
|2T1FF28P81C438164|Acura    |TL      |1956|Sport       |Used     |[]                                                    |
|1C4NJPFA8ED519678|Honda    |Civic   |2020|Sport       |Used     |[]                                                    |
|YV4960DZ8A2080987|Toyota   |null    |2019|Base        |Used     |[hasNotModel]                                         |
|3C3CFFAR3DT571989|Audi     |A4      |2022|300         |New      |[]                                                    |
|5FNYF4H5XFB054084|Ford     |F-150   |2025|null        |New      |[]                                                    |
|JM1BM1U79E1108676|Chevr

##### Validation (validYear)

In [83]:
heyauto_val6_df = heyauto_val5_df.withColumn("validationTest", 
                                                F.when(F.expr("year BETWEEN 1990 AND 2024 OR year is NULL"), F.array_union(F.col("validationTest"), F.array(F.lit("notValidYear"))))\
                                                .otherwise(F.col("validationTest")))

In [84]:
heyauto_val6_df.show(truncate=False)

+-----------------+---------+--------+----+------------+---------+--------------------------------------------------------------------+
|vin              |make     |model   |year|trim        |Condition|validationTest                                                      |
+-----------------+---------+--------+----+------------+---------+--------------------------------------------------------------------+
|2T1FF28P81C438164|Acura    |TL      |1956|Sport       |Used     |[]                                                                  |
|1C4NJPFA8ED519678|Honda    |Civic   |2020|Sport       |Used     |[notValidYear]                                                      |
|YV4960DZ8A2080987|Toyota   |null    |2019|Base        |Used     |[hasNotModel, notValidYear]                                         |
|3C3CFFAR3DT571989|Audi     |A4      |2022|300         |New      |[notValidYear]                                                      |
|5FNYF4H5XFB054084|Ford     |F-150   |2025|null 

### Return AcceptedVehicle and RejectedVehicle Table

In [85]:
accepted_auto_df = heyauto_val6_df.select(F.col("vin"))\
                                    .filter(F.size("validationTest") == 0)

In [86]:
accepted_auto_df.show(truncate=False)

+-----------------+
|vin              |
+-----------------+
|2T1FF28P81C438164|
|5FNYF4H5XFB054084|
+-----------------+



In [87]:
rejected_auto_df = heyauto_val6_df.select(F.col("vin"), F.col("validationTest").alias("failed"))\
                                    .filter(F.size("validationTest") != 0)

In [88]:
rejected_auto_df.show(truncate=False)

+-----------------+--------------------------------------------------------------------+
|vin              |failed                                                              |
+-----------------+--------------------------------------------------------------------+
|1C4NJPFA8ED519678|[notValidYear]                                                      |
|YV4960DZ8A2080987|[hasNotModel, notValidYear]                                         |
|3C3CFFAR3DT571989|[notValidYear]                                                      |
|JM1BM1U79E1108676|[notValidYear]                                                      |
|5TDZA23C46S533612|[hasNotYear, notValidYear]                                          |
|JTEBU5JR8A5024764|[notValidYear]                                                      |
|2G4WS52J121278965|[notValidYear]                                                      |
|5TDZA3EH4BS08656 |[notValidVin, notValidYear]                                         |
|2G4WS52J121278966|[h